## Author of this notebook : Ramsri Goutham Golla

Linkedin: https://www.linkedin.com/in/ramsrig/

Twitter : https://twitter.com/ramsri_goutham

## 1. Split a sentence at ending noun phrase or verb phrase

In [ ]:
!pip install --quiet allennlp==0.9.0
!pip install --quiet spacy==2.1.9
!pip install --quiet https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.1.0/en_core_web_sm-2.1.0.tar.gz

: 

In [10]:
!pip install overrides==3.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

from allennlp.predictors.predictor import Predictor

predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/elmo-constituency-parser-2018.03.14.tar.gz")

: 

In [ ]:
test_sentence = "The old woman was sitting under a tree and sipping coffee."
test_sentence = test_sentence.rstrip('?:!.,;')
print (test_sentence)
parser_output = predictor.predict(sentence=test_sentence)
print (parser_output)

Your label namespace was 'pos'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.


The old woman was sitting under a tree and sipping coffee
{'class_probabilities': [[1.0, 2.2363701168703898e-10, 2.1188745168967873e-15, 6.991575965094657e-15, 5.0569417021375206e-15, 3.1638940599376664e-12, 6.667722743294163e-14, 4.159836104050685e-12, 6.377996047968182e-11, 2.7916475554834452e-12, 6.635645944394822e-15, 2.2516380036516348e-13, 1.8319347150352883e-09, 5.116836926497177e-14, 1.8848717725505892e-11, 1.332261991698891e-12, 7.699951312949323e-15, 6.265735326416166e-12, 3.3666825837580516e-12, 2.0015255728852988e-13, 3.2307917192248015e-12, 1.226238693918802e-10, 9.285493485222673e-10, 3.863675059143784e-13, 8.740566186408658e-14, 1.0080882661836835e-13, 7.664675101201612e-13, 3.5253200098700477e-12, 2.2905963015023775e-14, 6.031352717608685e-13, 1.7653281614293803e-11, 4.304822817657872e-11, 2.169287839637324e-11, 4.3420917360779965e-13, 9.326480560067907e-13, 1.3978086049748484e-11, 4.3535772317922217e-13, 8.126962644516844e-13, 3.1398952448496997e-12, 7.4556333329312e-1

In [ ]:
tree_string = parser_output["trees"]
print (tree_string)

(S (NP (DT The) (JJ old) (NN woman)) (VP (VBD was) (VP (VP (VBG sitting) (PP (IN under) (NP (DT a) (NN tree)))) (CC and) (VP (VBG sipping) (NP (NN coffee))))))


In [ ]:
from nltk import tokenize
from nltk.tree import Tree

tree = Tree.fromstring(tree_string)
print (tree)
print (tree.pretty_print())

(S
  (NP (DT The) (JJ old) (NN woman))
  (VP
    (VBD was)
    (VP
      (VP (VBG sitting) (PP (IN under) (NP (DT a) (NN tree))))
      (CC and)
      (VP (VBG sipping) (NP (NN coffee))))))
                            S                                          
      ______________________|________                                   
     |                               VP                                
     |          _____________________|_______                           
     |         |                             VP                        
     |         |                  ___________|________________          
     |         |                 VP               |           |        
     |         |      ___________|___             |           |         
     |         |     |               PP           |           VP       
     |         |     |       ________|___         |      _____|____     
     NP        |     |      |            NP       |     |          NP  
  ___|____   


Links to understand syntactic parsing and related terms:

http://www.cs.cornell.edu/courses/cs474/2004fa/lec1.pdf

http://sameersingh.org/courses/statnlp/wi17/slides/lecture-0207-syntactic-parsing.pdf





S	sentence	

NP	noun phrase	

VP	verb phrase	

PP	prepositional phrase	

Det	determiner	

N	noun	

V	verb	

P	preposition	

VBD - Past Tense Verb

JJ - Adjective

etc

In [ ]:
tree.pretty_print()
temp1 = tree[0]
temp2 = tree[1]
temp3 = tree[-1]
temp1.pretty_print()
temp2.pretty_print()
temp3.pretty_print()

                            S                                          
      ______________________|________                                   
     |                               VP                                
     |          _____________________|_______                           
     |         |                             VP                        
     |         |                  ___________|________________          
     |         |                 VP               |           |        
     |         |      ___________|___             |           |         
     |         |     |               PP           |           VP       
     |         |     |       ________|___         |      _____|____     
     NP        |     |      |            NP       |     |          NP  
  ___|____     |     |      |         ___|___     |     |          |    
 DT  JJ   NN  VBD   VBG     IN       DT      NN   CC   VBG         NN  
 |   |    |    |     |      |        |       |    |     | 

In [ ]:
# split at right most nounphrase or verbphrase

def get_flattened(t):
    sent_str_final = None
    if t is not None:
        sent_str = [" ".join(x.leaves()) for x in list(t)]
        sent_str_final = [" ".join(sent_str)]
        sent_str_final = sent_str_final[0]
    return sent_str_final

def get_right_most_VP_or_NP(parse_tree,last_NP = None,last_VP = None):
    if len(parse_tree.leaves()) == 1:
        return last_NP,last_VP
    last_subtree = parse_tree[-1]
    if last_subtree.label() == "NP":
        last_NP = last_subtree
    elif last_subtree.label() == "VP":
        last_VP = last_subtree
    
    return get_right_most_VP_or_NP(last_subtree,last_NP,last_VP)


last_nounphrase, last_verbphrase =  get_right_most_VP_or_NP(tree)
last_nounphrase_flattened = get_flattened(last_nounphrase)
last_verbphrase_flattened = get_flattened(last_verbphrase)

print ("Original Sentence ",test_sentence)
print ("last_nounphrase ",last_nounphrase )
print ("last_verbphrase ",last_verbphrase)
print ("\n ")
print ("last_nounphrase ",last_nounphrase_flattened )
print ("last_verbphrase ",last_verbphrase_flattened)

Original Sentence  The old woman was sitting under a tree and sipping coffee
last_nounphrase  (NP (NN coffee))
last_verbphrase  (VP (VBG sipping) (NP (NN coffee)))

 
last_nounphrase  coffee
last_verbphrase  sipping coffee


In [ ]:
import re

# sub_string - sipping coffee
# main_string - The old woman was sitting under a tree and sipping coffee
# compare like below
# Theoldwomanwassittingunderatreeandsippingcoffee  || sippingcoffee
# oldwomanwassittingunderatreeandsippingcoffee || sippingcoffee
# womanwassittingunderatreeandsippingcoffee || sippingcoffee
# ...............
# andsippingcoffee || sippingcoffee
# sippingcoffee || sippingcoffee
def get_termination_portion(main_string, sub_string):
    combined_sub_string = sub_string.replace(" ", "")
    main_string_list = main_string.split()
    last_index = len(main_string_list)
    for i in range(last_index):
        check_string_list = main_string_list[i:]
        check_string = "".join(check_string_list)
        check_string = check_string.replace(" ", "")
        if check_string == combined_sub_string:
            return " ".join(main_string_list[:i])

    return None

longest_phrase_to_use = max(last_nounphrase_flattened, last_verbphrase_flattened)
print ("Ending phrase: ", longest_phrase_to_use)

longest_phrase_to_use = re.sub(r"-LRB- ", "(", longest_phrase_to_use)
longest_phrase_to_use = re.sub(r" -RRB-", ")", longest_phrase_to_use)


split_sentence = get_termination_portion(test_sentence, longest_phrase_to_use)
print ("Original sentence : ",test_sentence)
print ("Original sentence after splitting at ending phrase: ",split_sentence)

Ending phrase:  sipping coffee
Original sentence :  The old woman was sitting under a tree and sipping coffee
Original sentence after splitting at ending phrase:  The old woman was sitting under a tree and


**Another example to split at first noun phrase or verb phrase**

In [ ]:
# split at the first noun phrase or verb phrase

test_sentence2 = "They had no ice cream left at home, nor did they have money to go to the store."
test_sentence2 = test_sentence2.rstrip('?:!.,;')
print (test_sentence2)
parser_output2 = predictor.predict(sentence=test_sentence2)
tree_string2 = parser_output2["trees"]

tree2 = Tree.fromstring(tree_string2)
print (tree2.pretty_print())


In [ ]:

# SBAR stands for Subordinate Clause.
#  Penn Tree bank overview - http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.9.8216&rep=rep1&type=pdf

def get_first_VP_and_NP_and_sentence(parse_tree, first_NP=None, first_VP=None, first_sent=None):
    if len(parse_tree.leaves()) == 1:
        return get_flattened(first_NP), get_flattened(first_VP), get_flattened(first_sent)
    last_subtree = parse_tree[-1]

    if last_subtree.label() == "NP" and not first_NP:
        first_NP = last_subtree
    elif last_subtree.label() == "VP" and not first_VP:
        first_VP = last_subtree
    elif last_subtree.label() == "S" and not first_sent:
        first_sent = last_subtree

    return get_first_VP_and_NP_and_sentence(last_subtree, first_NP, first_VP, first_sent)


first_nounphrase, first_verbphrase, first_sentence = get_first_VP_and_NP_and_sentence(tree2)

print("first_nounphrase: ",first_nounphrase)
print ("first_verbphrase: ",first_verbphrase)
print ("first_sentence: ",first_sentence)


In [ ]:
longest_phrase_to_use = max(first_nounphrase, first_verbphrase)
print ("Ending phrase: ", longest_phrase_to_use)

longest_phrase_to_use = re.sub(r"-LRB- ", "(", longest_phrase_to_use)
longest_phrase_to_use = re.sub(r" -RRB-", ")", longest_phrase_to_use)


split_sentence = get_termination_portion(test_sentence2, longest_phrase_to_use)
print ("Original sentence : ",test_sentence2)
print ("Original sentence after splitting at ending phrase: ",split_sentence)

![picture](https://drive.google.com/uc?export=view&id=1u1xuSJ335jtc9CzC_t7AIakUgfZCK2qO)



## 2. Generate alternate endings to a split sentence using GPT-2

In [ ]:
!pip install --quiet transformers==2.8.0

     |████████████████████████████████| 573kB 8.5MB/s 
     |████████████████████████████████| 5.6MB 20.3MB/s 
     |████████████████████████████████| 890kB 41.3MB/s 


In [ ]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

# GPT2tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
# GPT2model = TFGPT2LMHeadModel.from_pretrained("distilgpt2",pad_token_id=GPT2tokenizer.eos_token_id)
GPT2tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
GPT2model = TFGPT2LMHeadModel.from_pretrained("gpt2",pad_token_id=GPT2tokenizer.eos_token_id)


In [ ]:
partial_sentence = "The old woman was sitting under a tree and"
input_ids = GPT2tokenizer.encode(partial_sentence,return_tensors='tf')
print (input_ids)
maximum_length = len(partial_sentence.split())+40

tf.Tensor([[ 464 1468 2415  373 5586  739  257 5509  290]], shape=(1, 9), dtype=int32)


In [ ]:
# Activate top_k sampling and top_p sampling with only from 90% most likely words
sample_outputs = GPT2model.generate(
    input_ids, 
    do_sample=True, 
    max_length=maximum_length, 
    top_p=0.80, # 0.85 
    top_k=60,   #30
    repetition_penalty  = 10.0,
    num_return_sequences=12
)

In [ ]:
import nltk
nltk.download('punkt')
from nltk import tokenize
generated_sentences=[]

for i, sample_output in enumerate(sample_outputs):
    decoded_sentence = GPT2tokenizer.decode(sample_output, skip_special_tokens=True)
    # final_sentence = decoded_sentence
    final_sentence = tokenize.sent_tokenize(decoded_sentence)[0]
    generated_sentences.append(final_sentence)
    print (i,": ",final_sentence)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
0 :  The old woman was sitting under a tree and looking out the window, watching them.
1 :  The old woman was sitting under a tree and I started yelling at her, 'Do you know what happens if your wife dies?
2 :  The old woman was sitting under a tree and the young man, holding her hand in his own.
3 :  The old woman was sitting under a tree and watching as the boy's hair started to fall off.
4 :  The old woman was sitting under a tree and trying to calm herself by saying "Oh my God."
5 :  The old woman was sitting under a tree and could see the man coming.
6 :  The old woman was sitting under a tree and the man came up behind her.
7 :  The old woman was sitting under a tree and when the light came, she fell in love with me."
8 :  The old woman was sitting under a tree and he held the sword in her hand.
9 :  The old woman was sitting under a tree and looking at the window, when she h